# read_tmdb_data.ipynb

This notebook reads JSON data from TMDB's API and stores it as CSV files:
- **movies.csv**: Data on approx. 27000 movies from the United States from 2000-2023.
- **persons.csv**: Data on the directors and top 10 cast members in these movies.

In [1]:
import os
import requests
from dotenv import load_dotenv
from time import sleep
import pandas as pd

load_dotenv()
tmdb_api_token = os.getenv("TMDB_API_TOKEN")

headers = {
    "accept": "application/json",
    "Authorization": f"Bearer {tmdb_api_token}"
}

REQUEST_DELAY_SECONDS = 0.02

First we request movie_ids for all movies that live up to these requirements:
- From the United States
- From the years 2000-2023
- With original_langauge = english
- With TMDB vote count ≥ 10

In [ ]:
base_url = (
    "https://api.themoviedb.org/3/discover/movie"
    "?include_adult=false"
    "&include_video=false"
    "&with_origin_country=US"
    "&with_original_language=en"
    "&vote_count.gte=10"
    "&sort_by=primary_release_date.asc"
)

movie_ids = []

# Loop through the years
for year in range(2000, 2024):

    # Loop through the pages (TMDB uses a maximum of 500 pages)
    for page in range(1, 501):
        url = f"{base_url}&primary_release_year={year}&page={page}"
        response = requests.get(url, headers=headers)
        if response.status_code != 200:
            raise Exception(f"year: {year} page: {page} status_code: {response.status_code} text: {response.text}")
    
        # Extract movie_results from the response
        movie_results = response.json().get("results") 
    
        # Stop if we have reached the last page
        if not movie_results:
            break

        # Extend movie_ids list with the ones from movie_results
        movie_ids.extend([movie["id"] for movie in movie_results])
    
        sleep(REQUEST_DELAY_SECONDS)

print(f"Number of movie ids found: {len(movie_ids)}")

Then we request movie data for all these movie_ids, and store that in a dataframe and a CSV file

In [ ]:
all_movies = []

# Loop through all movie_ids and request movie data
for movie_id in movie_ids:
    url = f"https://api.themoviedb.org/3/movie/{movie_id}?append_to_response=credits"
    response = requests.get(url, headers=headers)
    if response.status_code != 200:
        raise Exception(f"movie_id: {movie_id} status_code: {response.status_code} text: {response.text}")
    
    # Extract the movie data from the response as a dictionary
    movie = response.json()

    # Find the directors and put director_person_ids in the dictionary
    movie["director_person_ids"] = []
    for credit in movie["credits"]["crew"]:
        if(credit["job"]=="Director"):
            movie["director_person_ids"].append(credit["id"])

    # Simplify child dictionaries to list of ids
    movie["genre_ids"] = [genre["id"] for genre in movie["genres"]]
    movie["spoken_languages"] = [language["iso_639_1"] for language in movie["spoken_languages"]]
    movie["production_company_ids"] = [company["id"] for company in movie["production_companies"]]
    movie["production_countries"] = [country["iso_3166_1"] for country in movie["production_countries"]]
    movie["collection_id"] = movie["belongs_to_collection"]["id"] if movie["belongs_to_collection"] else pd.NA
    movie["cast_person_ids"] = [cast_member["id"] for cast_member in movie["credits"]["cast"]]
    movie["cast_credit_ids"] = [cast_member["credit_id"] for cast_member in movie["credits"]["cast"]]
    movie["crew_person_ids"] = [crew_member["id"] for crew_member in movie["credits"]["crew"]]
    movie["crew_credit_ids"] = [crew_member["credit_id"] for crew_member in movie["credits"]["crew"]]
    del movie['genres']
    del movie['production_companies']
    del movie["belongs_to_collection"]   
    del movie["credits"]
    
    all_movies.append(movie)
    sleep(REQUEST_DELAY_SECONDS)  

df_movies = pd.DataFrame(all_movies)
df_movies.rename(columns={"id": "movie_id"}, inplace=True)
df_movies.to_csv("../movie_data/movies.csv", index=False)
print(f"Number of movies collected: {len(df_movies)}")
df_movies

Now we find person ids for the directors in each of the movies.

And then we find person ids for the top 10 cast members in each of the movies.

And we combine these two lists to a unique_person_ids set.

In [2]:
import ast  # Abstract Syntax Trees - safely parse string list to Python list
df_movies = pd.read_csv("../movie_data/movies.csv")
df_movies['director_person_ids'] = df_movies['director_person_ids'].apply(ast.literal_eval) # Turn director_person_ids into actual Python list
df_movies['cast_person_ids'] = df_movies['cast_person_ids'].apply(ast.literal_eval) # Turn cast_person_ids into actual Python list
df_movies

,adult,backdrop_path,budget,homepage,movie_id,imdb_id,origin_country,original_language,original_title,overview,...,vote_average,vote_count,director_person_ids,genre_ids,production_company_ids,collection_id,cast_person_ids,cast_credit_ids,crew_person_ids,crew_credit_ids
0,False,/dD90r6NQ8cFgYjjYGSLRQLCdJWN.jpg,0,NaN,515728,tt0191181,['US'],en,Hitch,Two friends are on a road trip and a one-sided...,...,4.900,10,[131388],[18],[],NaN,"[1230580, 2030046]","['5ac28e640e0a260c140239f0', '5ae16159c3a36876...","[1434896, 131388, 131388, 131388, 131388, 1360...","['6454f309c044290143e43376', '6454f31187a27a01..."
1,False,/ifq88qw3vgoKlUyw0OAmPQCSqBc.jpg,0,NaN,300236,tt0259233,['US'],en,Carnage: The Legend of Quiltface,Four students set out for the barren Nevada de...,...,2.500,10,[103123],[27],"[4708, 110668]",NaN,"[98740, 99106, 98276, 1685427, 1771744, 177174...","['58bf16e6925141608406b270', '6251f3b9a6c10432...","[103123, 98868, 103123, 1001648, 103123, 10016...","['62c5bceaf794ad00bf5a8867', '62c5bce19ba86a00..."
2,False,/c5UlEYHM2xuSrfxESg1gZSpVAEB.jpg,0,NaN,96716,tt0128977,['US'],en,The Bumblebee Flies Anyway,An amnesiac youth tries to piece together his ...,...,6.200,30,[126537],"[18, 10749]","[1596, 1363]",NaN,"[109, 21197, 16407, 1223778, 38581, 303197, 56...","['52fe49be9251416c750d1f8f', '52fe49be9251416c...","[1534680, 2556479, 1516275, 68126, 1516278, 27...","['60c1e4cb39a45d0040c5cd4b', '60c1e503960cde00..."
3,False,NaN,0,NaN,71618,tt0198284,['US'],en,After Sex,A group of attractive women get together for a...,...,5.000,27,[176312],"[35, 18, 10749]","[85165, 86531]",NaN,"[170638, 12519, 15110, 3208, 51670, 61962, 117...","['53d9db080e0a2652f0001583', '52fe483ec3a36848...","[176312, 1470931, 33008, 17210, 17211, 954441,...","['52fe483ec3a368484e0ef37f', '5564e59bc3a36874..."
4,False,/nKD4M8Oyuh5aE9EBR29A9WnJKxE.jpg,0,NaN,66131,tt0346794,['US'],en,A Constant Forge,"One of the great mavericks of cinema, John Cas...",...,6.600,13,[544690],[99],[],NaN,"[11147, 5950, 10556, 10127, 856, 2314, 1629458...","['5a9dfc9e0e0a2671fb009de8', '5a9dfc11c3a36842...","[544690, 3399184, 544690, 961119, 3399183, 339...","['52fe472bc3a368484e0b89d9', '61f15d7dcd204600..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27289,False,/gub9e0qRZWmmzTV77A6T95HELIF.jpg,40000000,https://www.mgm.com/movies/the-boys-in-the-boat,823452,tt1856080,['US'],en,The Boys in the Boat,The triumphant underdog story of the Universit...,...,7.233,311,[1461],"[18, 36]","[1782, 143790, 210099]",NaN,"[33192, 1371041, 27172, 1522758, 1780950, 2725...","['6205b7c5ce5d8200c83376f3', '61c21146904f6d00...","[3069889, 1461, 17148, 1500871, 1578875, 18195...","['6088693d84448e007932a40c', '6088695754a09800..."
27290,False,/rSle4ix2IZ6UMgCDY8L3ngdOmSZ.jpg,90000000,https://www.warnerbros.com/movies/color-purple,558915,tt1200263,['US'],en,The Color Purple,A decades-spanning tale of love and resilience...,...,7.049,326,[1507725],[18],"[174, 3298, 56, 150841, 200643, 216687]",NaN,"[165909, 40036, 1075037, 91671, 1154054, 34767...","['61fc8db841429100a2189044', '61f966bfeee18600...","[3511353, 4445624, 4445622, 73350, 2167926, 21...","['62b89b15a61de103b61eed9b', '65876f514772155a..."
27291,False,/vhTsS3K79lsyZNZ7t33IQiSREsO.jpg,0,https://www.netflix.com/title/81449757,1215278,tt30225680,"['US', 'GB']",en,Hell Camp: Teen Nightmare,Out-of-control teens across America were sent ...,...,5.663,46,[2283876],[99],[26359],NaN,"[5332322, 38406]","['67e5d2703e65c8ea88ba1e0f', '67e60712421eb8c3...","[2283876, 3248862, 2060153, 4208975, 1414935, ...","['65734ffc1c635b00c3a9bfbe', '67e5d17f421eb8c3..."
27292,False,/zS1OiCw3opLQYxjmhqhOQ7D7YyD.jpg,0,NaN,1156189,tt28490873,['US'],en,Ryuichi Sakamoto: Opus,"""Ars longa, vita brevis"" – art is long, life i...",...,7.861,18,[2010321],"[10402, 99]","[188205, 11561, 4667, 46298]",NaN,[11382],['64bfde5d8c0a4800aeefb198'],"[3056, 1610252, 2247870, 3406563, 4240016, 152...","['64ec9e951feac100fe5e313e', '64ec9e33

In [3]:
director_person_ids = df_movies['director_person_ids'].explode().dropna().tolist()

top_10_cast_person_ids = (
    df_movies['cast_person_ids']
    .apply(lambda ids: ids[:10])   # get first 10 for each movie
    .explode()                     # flatten into one Series
    .dropna()
    .tolist()                      # convert to plain list
)

unique_person_ids = set(director_person_ids) | set(top_10_cast_person_ids)
len(unique_person_ids)

104929

Finally we request person data for all these unique_person_ids, and store that in a dataframe and a CSV file.

In [4]:
all_persons = []

for person_id in unique_person_ids:
    url = f"https://api.themoviedb.org/3/person/{person_id}"
    response = requests.get(url, headers=headers)
    if response.status_code != 200:
        raise Exception(f"person_id: {person_id} status_code: {response.status_code} text: {response.text}")
    
    # Extract the person data from the response
    person = response.json()
    
    all_persons.append(person)
    sleep(REQUEST_DELAY_SECONDS)
    
    
df_persons = pd.DataFrame(all_persons)
df_persons.rename(columns={"id": "person_id"}, inplace=True)
df_persons.to_csv("../movie_data/persons.csv", index=False)
print(f"Number of persons collected: {len(df_persons)}")
df_persons

Exception: person_id: 1799632 status_code: 404 text: {"success":false,"status_code":34,"status_message":"The resource you requested could not be found."}

In [5]:
len(all_persons)

94154

In [11]:
all_persons[6445]["id"]

12523

In [13]:
person_ids_set = {person["id"] for person in all_persons}

In [14]:
len(person_ids_set)

94154

In [15]:
person_ids_missing = unique_person_ids - person_ids_set

In [16]:
len(person_ids_missing)

10775

In [17]:
person_ids_missing.discard(1799632)
len(person_ids_missing)

10774

In [18]:
for person_id in person_ids_missing:
    url = f"https://api.themoviedb.org/3/person/{person_id}"
    response = requests.get(url, headers=headers)
    if response.status_code != 200:
        raise Exception(f"person_id: {person_id} status_code: {response.status_code} text: {response.text}")
    
    # Extract the person data from the response
    person = response.json()
    
    all_persons.append(person)
    sleep(REQUEST_DELAY_SECONDS)
    
    
df_persons = pd.DataFrame(all_persons)
df_persons.rename(columns={"id": "person_id"}, inplace=True)
df_persons.to_csv("../movie_data/persons.csv", index=False)
print(f"Number of persons collected: {len(df_persons)}")
df_persons

Number of persons collected: 104928


,adult,also_known_as,biography,birthday,deathday,gender,homepage,person_id,imdb_id,known_for_department,name,place_of_birth,popularity,profile_path
0,False,"[George Walton Lucas Jr. , George Walton Lucas...","George Walton Lucas Jr. (born May 14, 1944) is...",1944-05-14,None,2,None,1,nm0000184,Directing,George Lucas,"Modesto, California, USA",1.5338,/mDLDvsx8PaZoEThkBdyaG1JxPdf.jpg
1,False,"[Mark Hamil, Mark Richard Hamill, Patrick Will...","Mark Richard Hamill (born September 25, 1951) ...",1951-09-25,None,2,None,2,nm0000434,Acting,Mark Hamill,"Oakland, California, USA",2.5029,/2ZulC2Ccq1yv3pemusks6Zlfy2s.jpg
2,False,[Harrison J. Ford],Legendary Hollywood Icon Harrison Ford was bor...,1942-07-13,None,2,None,3,nm0000148,Acting,Harrison Ford,"Chicago, Illinois, USA",3.4892,/zVnHagUvXkR2StdOtquEwsiwSVt.jpg
3,False,"[Carrie Frances Fisher , Кэрри Фрэнсис Фишер, ...","Carrie Frances Fisher (October 21, 1956 – Dece...",1956-10-21,2016-12-27,1,https://carriefisher.com/,4,nm0000402,Acting,Carrie Fisher,"Beverly Hills, Los Angeles, California, USA",1.2394,/awb4UqzT6meD3JiQlraIzAqcRtH.jpg
4,False,[],,None,None,2,None,1835010,nm0529183,Acting,Thomas Lyons,None,0.0886,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
104923,False,[Kristen],,None,None,1,None,1015800,nm3691901,Acting,Kristen Doute,"Dearborn, Michicagon, USA",0.0866,/wCXQ4cpl6Qb3ZjiaZh17PJJ4BkV.jpg
104924,False,[],,None,None,2,None,1048560,nm0578485,Acting,Johnny Melville,None,0.1834,/6G4ZDTDByPS2bF711ZLe8StiodR.jpg
104925,False,[Eric L. da Silva],,1977-06-30,None,2,None,1048570,nm0196230,Acting,Eric da Silva,"Homestead, Florida, USA",1.1655,/c7VoexEHz40xNkKxiYOpIa9GH05.jpg
104926,False,[David A. Kaye],Vancouver-based artist David Andrew Kaye was b...,1988-08-04,None,2,None,1048574,nm0443287,Acting,David Kaye,"Saskatoon, Saskatchewan, Canada",1.0889,/fAK9KpVtPeZ9MTUXamfVqwUGBId.jpg


In [19]:
# We need to remove bad person id from the df_movies data frame

In [20]:
df_movies['crew_person_ids'] = df_movies['crew_person_ids'].apply(ast.literal_eval) # Turn crew_person_ids into actual Python list

In [24]:
# Assuming df_movie is your DataFrame and the ID is a string
person_id = 1799632

# Find rows where the ID is in either cast_person_ids or crew_person_ids
mask = df_movies["cast_person_ids"].apply(lambda x: person_id in x) | \
       df_movies["crew_person_ids"].apply(lambda x: person_id in x)

# Filter the DataFrame
filtered_df = df_movies[mask]
filtered_df

,adult,backdrop_path,budget,homepage,movie_id,imdb_id,origin_country,original_language,original_title,overview,...,vote_average,vote_count,director_person_ids,genre_ids,production_company_ids,collection_id,cast_person_ids,cast_credit_ids,crew_person_ids,crew_credit_ids
11916,False,/wBUmTf1Yu2Tn8eGD7UPaiSrzgBI.jpg,0,http://www.siriusdisclosure.com,186254,tt2637848,['US'],en,Sirius,Dr. Steven Greer—an Emergency room doctor turn...,...,6.200,24,"[544864, 1168681]",[99],[27005],NaN,"[1799632, 11155, 2093349, 120575, 21111, 12057...","['5e9bdc16371097001c47afae', '5e9bdc8937109700...","[544864, 1168681, 2606751, 544864, 2606753, 26...","['52fe4cf79251416c7512b881', '52fe4cf79251416c..."
18206,False,/wj0xfNvCbXB4RXIMcsla50fhz35.jpg,0,http://www.siriusdisclosure.com,450594,tt6400614,['US'],en,Unacknowledged,Dr. Steven Greer presents brand new top-secret...,...,6.273,139,[1789716],"[99, 9648, 14, 878]",[89148],NaN,"[4808, 1799632]","['58e3eb48c3a36872f6021d26', '58f807019251415b...","[1789716, 1789717, 1789716, 1789717, 1799633, ...","['58e3eb2f92514127f6022bf7', '58e3eb3ac3a36873..."
22863,False,/efiDNmExd9GweIqmQKcwIt09hi7.jpg,0,https://ce5film.com/,688335,tt12108272,['US'],en,Close Encounters of the Fifth Kind,"Dr. Steven Greer’s previous works, SIRIUS and ...",...,7.000,90,[1789716],[99],[126911],NaN,"[12799, 1799632, 1591669]","['5e9b100dfd6fa1001d01f477', '5e9b102331032500...","[2180034, 1789716, 1799633, 1789716, 2606161, ...","['63bab5a8a6e2d2008b3deeae', '5e9b13ffeb79c200..."
26690,False,/kbuyKwU8pyPR83J4kr0nBIyquGL.jpg,500000,https://thelostcenturyfilm.com,1136906,tt27680530,['US'],en,The Lost Century: And How to Reclaim It,A century of illegal UFO secrecy has cost huma...,...,6.100,11,[1789716],[99],[147303],NaN,"[1799632, 2721327, 29783, 3262369]","['685c6f35e71fdd0f387b3c58', '647f65b00fb39800...","[1789716, 2606161, 1799632, 1789716, 1799632, ...","['647f65facaef2d0119c182e2', '647f6616caef2d00..."


In [26]:
movie_ids_to_drop = [186254, 450594, 688335, 1136906]
df_movies = df_movies[~df_movies['movie_id'].isin(movie_ids_to_drop)].reset_index(drop=True)
len(df_movies)

27290

In [27]:
df_movies.to_csv("../movie_data/movies.csv", index=False)
print(f"Number of movies collected: {len(df_movies)}")
df_movies

Number of movies collected: 27290


,adult,backdrop_path,budget,homepage,movie_id,imdb_id,origin_country,original_language,original_title,overview,...,vote_average,vote_count,director_person_ids,genre_ids,production_company_ids,collection_id,cast_person_ids,cast_credit_ids,crew_person_ids,crew_credit_ids
0,False,/dD90r6NQ8cFgYjjYGSLRQLCdJWN.jpg,0,NaN,515728,tt0191181,['US'],en,Hitch,Two friends are on a road trip and a one-sided...,...,4.900,10,[131388],[18],[],NaN,"[1230580, 2030046]","['5ac28e640e0a260c140239f0', '5ae16159c3a36876...","[1434896, 131388, 131388, 131388, 131388, 1360...","['6454f309c044290143e43376', '6454f31187a27a01..."
1,False,/ifq88qw3vgoKlUyw0OAmPQCSqBc.jpg,0,NaN,300236,tt0259233,['US'],en,Carnage: The Legend of Quiltface,Four students set out for the barren Nevada de...,...,2.500,10,[103123],[27],"[4708, 110668]",NaN,"[98740, 99106, 98276, 1685427, 1771744, 177174...","['58bf16e6925141608406b270', '6251f3b9a6c10432...","[103123, 98868, 103123, 1001648, 103123, 10016...","['62c5bceaf794ad00bf5a8867', '62c5bce19ba86a00..."
2,False,/c5UlEYHM2xuSrfxESg1gZSpVAEB.jpg,0,NaN,96716,tt0128977,['US'],en,The Bumblebee Flies Anyway,An amnesiac youth tries to piece together his ...,...,6.200,30,[126537],"[18, 10749]","[1596, 1363]",NaN,"[109, 21197, 16407, 1223778, 38581, 303197, 56...","['52fe49be9251416c750d1f8f', '52fe49be9251416c...","[1534680, 2556479, 1516275, 68126, 1516278, 27...","['60c1e4cb39a45d0040c5cd4b', '60c1e503960cde00..."
3,False,NaN,0,NaN,71618,tt0198284,['US'],en,After Sex,A group of attractive women get together for a...,...,5.000,27,[176312],"[35, 18, 10749]","[85165, 86531]",NaN,"[170638, 12519, 15110, 3208, 51670, 61962, 117...","['53d9db080e0a2652f0001583', '52fe483ec3a36848...","[176312, 1470931, 33008, 17210, 17211, 954441,...","['52fe483ec3a368484e0ef37f', '5564e59bc3a36874..."
4,False,/nKD4M8Oyuh5aE9EBR29A9WnJKxE.jpg,0,NaN,66131,tt0346794,['US'],en,A Constant Forge,"One of the great mavericks of cinema, John Cas...",...,6.600,13,[544690],[99],[],NaN,"[11147, 5950, 10556, 10127, 856, 2314, 1629458...","['5a9dfc9e0e0a2671fb009de8', '5a9dfc11c3a36842...","[544690, 3399184, 544690, 961119, 3399183, 339...","['52fe472bc3a368484e0b89d9', '61f15d7dcd204600..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27285,False,/gub9e0qRZWmmzTV77A6T95HELIF.jpg,40000000,https://www.mgm.com/movies/the-boys-in-the-boat,823452,tt1856080,['US'],en,The Boys in the Boat,The triumphant underdog story of the Universit...,...,7.233,311,[1461],"[18, 36]","[1782, 143790, 210099]",NaN,"[33192, 1371041, 27172, 1522758, 1780950, 2725...","['6205b7c5ce5d8200c83376f3', '61c21146904f6d00...","[3069889, 1461, 17148, 1500871, 1578875, 18195...","['6088693d84448e007932a40c', '6088695754a09800..."
27286,False,/rSle4ix2IZ6UMgCDY8L3ngdOmSZ.jpg,90000000,https://www.warnerbros.com/movies/color-purple,558915,tt1200263,['US'],en,The Color Purple,A decades-spanning tale of love and resilience...,...,7.049,326,[1507725],[18],"[174, 3298, 56, 150841, 200643, 216687]",NaN,"[165909, 40036, 1075037, 91671, 1154054, 34767...","['61fc8db841429100a2189044', '61f966bfeee18600...","[3511353, 4445624, 4445622, 73350, 2167926, 21...","['62b89b15a61de103b61eed9b', '65876f514772155a..."
27287,False,/vhTsS3K79lsyZNZ7t33IQiSREsO.jpg,0,https://www.netflix.com/title/81449757,1215278,tt30225680,"['US', 'GB']",en,Hell Camp: Teen Nightmare,Out-of-control teens across America were sent ...,...,5.663,46,[2283876],[99],[26359],NaN,"[5332322, 38406]","['67e5d2703e65c8ea88ba1e0f', '67e60712421eb8c3...","[2283876, 3248862, 2060153, 4208975, 1414935, ...","['65734ffc1c635b00c3a9bfbe', '67e5d17f421eb8c3..."
27288,False,/zS1OiCw3opLQYxjmhqhOQ7D7YyD.jpg,0,NaN,1156189,tt28490873,['US'],en,Ryuichi Sakamoto: Opus,"""Ars longa, vita brevis"" – art is long, life i...",...,7.861,18,[2010321],"[10402, 99]","[188205, 11561, 4667, 46298]",NaN,[11382],['64bfde5d8c0a4800aeefb198'],"[3056, 1610252, 2247870, 3406563, 4240016, 152...","['64ec9e951feac100fe5e313e', '64ec9e33